In [36]:
import os
import json
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

## AutoFL

In [31]:
def extract_length(trajectory):
    return len(trajectory)

def extract_function_call_counts(trajectory):
    return len([m for m in trajectory if "function_call" in m])

def extract_content_length(trajectory):
    return len(''.join([m['content'] for m in trajectory if m['role'] == 'assistant' and m['content']]))

def is_found(buggy_methods):
    return any([buggy_methods[m]['is_found'] for m in buggy_methods])

In [38]:
AUTOFL_DIR = Path('raw_data/autofl')

results = list()
for run_dir in tqdm(os.listdir(AUTOFL_DIR)):
    if not os.path.isdir(AUTOFL_DIR / run_dir):
        continue
    run_id = int(run_dir.split('_')[-1])
    for model_dir in os.listdir(AUTOFL_DIR / run_dir):
        assert os.path.isdir(AUTOFL_DIR / run_dir / model_dir)
        for file in os.listdir(AUTOFL_DIR / run_dir / model_dir):
            assert 'json' in file
            bug_id = file.split('-')[1].split('.')[0]
            with open(AUTOFL_DIR / run_dir / model_dir / file) as f:
                data = json.load(f)
                if 'messages' not in data or type(data['messages']) != list:
                    continue
                if 'buggy_methods' not in data or type(data['buggy_methods']) != dict:
                    continue
                trajectory = data['messages']
                results.append({
                    'bug_id': bug_id,
                    'run_id': run_id,
                    'model': model_dir,
                    'length': extract_length(trajectory),
                    'function_call_count': extract_function_call_counts(trajectory),
                    'content_length': extract_content_length(trajectory),
                    'success': is_found(data['buggy_methods']),
                })


print(len(results))

100%|██████████| 21/21 [00:17<00:00,  1.17it/s]

32308


In [39]:
df = pd.DataFrame(results)
df.to_csv('data/autofl_base.csv')